# 本文件说明
- 图片识别

# 基本设置

In [1]:
import os
import base64
import hashlib  
import time  
import random  
import string  
from urllib.parse import quote  
import requests  

In [2]:
def curlmd5(src):  
    m = hashlib.md5(src.encode('UTF-8'))
    return m.hexdigest().upper()
 
# 请求时间戳（秒级），用于防止请求重放（保证签名5分钟有效） 
def get_params(base64_data):
    t = time.time()
    time_stamp=str(int(t))  
    # 请求随机字符串，用于保证签名不可预测  
    nonce_str = ''.join(random.sample(string.ascii_letters + string.digits, 10))  
    #print(nonce_str)
    # 应用标志，这里修改成自己的id和key  
    app_id = '2107878067'  
    app_key = '1GhxSLrCfQydpL81'
    params = {'app_id':app_id,  
              'image':base64_data,
              'time_stamp':time_stamp,
              'nonce_str':nonce_str,
              #'format':'1',
              #'topk':'5',
             }
    sign_before = ''
    # 要对key排序再拼接
    for key in sorted(params):
        # 键值拼接过程value部分需要URL编码，URL编码算法用大写字母，例如%E8。quote默认大写。  
        sign_before += '{}={}&'.format(key,quote(params[key], safe=''))
        #print(quote(params[key], safe=''))
    # 将应用密钥以app_key为键名，拼接到字符串sign_before末尾  
    sign_before += 'app_key={}'.format(app_key)
    # 对字符串sign_before进行MD5运算，得到接口请求签名
    #print(sign_before)
    sign = curlmd5(sign_before)
    params['sign'] = sign
    #print(sign)
    #print(params)
    return params

# 识别图片

In [26]:
name_list = os.listdir('data/')
name_list

['河南_APP.png',
 '河南_公众号.png',
 '河南_网站.png',
 '湖北_APP.png',
 '湖北_公众号.png',
 '湖北_网站.png']

In [27]:
url = "https://api.ai.qq.com/fcgi-bin/ocr/ocr_generalocr"
#url = "https://api.ai.qq.com/fcgi-bin/vision/vision_objectr"

res = []

for name in name_list:
    p = name.split('_')[0]
    c = name.split('.')[0].split('_')[1]
    with open('data/{0}'.format(name),'rb') as fin:
        image_data = fin.read()
    base64_data = base64.b64encode(image_data)
    params = get_params(base64_data)
    #print(params)
    r = requests.post(url,data=params)
    item_list = r.json()['data']['item_list']
    #print(r.json())
    
    tmp_res = []
    for index, s in enumerate(item_list):
        if index > 3:
            tmp_res.append(s['itemstring'].replace(' ',''))
            if (index+1) % 4 == 0:
    #             print(s['itemstring'])
                res.append([p,c] + tmp_res)
                tmp_res = []
            

In [28]:
res

[['河南', 'APP', '1', '“大河网”', '大河网络传媒集团有限公司', '41120170001'],
 ['河南', 'APP', '2', '“眼遇”', '大河网络传媒集团有限公司', '41120170001'],
 ['河南', 'APP', '3', '“掌上洛阳”', '洛阳日报社', '41120170003'],
 ['河南', 'APP', '4', '“汴梁风”', '开封日报社', '41120170004'],
 ['河南', 'APP', '5', '“掌上开封”', '开封日报社', '41120170004'],
 ['河南', 'APP', '6', '“映象新闻客户端”', '河南广播电视台', '41120170005'],
 ['河南', '公众号', '“大河网”，腾讯微信', '大河网络传媒集团有限公司', '41120170001', '公众账号。'],
 ['河南', '公众号', '“大河网”,新浪微博', '大河网络传媒集团有限公司', '41120170001', '公众账号。'],
 ['河南', '公众号', '“中原网”，腾讯微信', '郑州报业集团', '41120170002', '公众账号。'],
 ['河南', '公众号', '“中原网”,新浪微博', '郑州报业集团', '41120170002', '公众账号。'],
 ['河南', '公众号', '“洛阳网”，腾讯微信', '洛阳日报社', '41120170003', '公众账号。'],
 ['河南', '公众号', '“洛阳网”，新浪微博', '洛阳日报社', '41120170003', '公众账号。'],
 ['河南', '公众号', '“开封网”，腾讯微信', '开封日报社', '41120170004', '公众账号。'],
 ['河南', '公众号', '8', '“开封网”，新浪微博', '开封日报社', '41120170004'],
 ['河南', '公众号', '公众账号。', '9', '“映象网”，腾讯微信', '河南广播电视台'],
 ['河南', '公众号', '41120170005', '公众账号。', '10', '“映象网”，新浪微博'],
 ['河南', '网站', '1', '大河网'

In [30]:
import pandas as pd
pd.DataFrame(res).to_excel('图片识别结果.xlsx')